In [1]:
from peewee import (
    SqliteDatabase, JOIN, 
    Model, Field, 
    AutoField, ForeignKeyField, 
    TextField, FloatField, 
    IntegerField, BooleanField, 
    TimeField
)

In [2]:
db = SqliteDatabase("test_db.db", pragmas = { "foreign_keys" : 1, "journal_mode": "wal" })

In [3]:
class ListField(Field):
    field_type = "list"

    def db_value(self, value):
        return str(value)

    def python_value(self, value):
        pval = []

        for entry in value[1 : -1].split(","):
            try:
                pval.append(float(entry))

            except:
                pval.append(entry.strip().replace("'", ""))

        return pval

In [4]:
class Structure(Model):
    structure_id = AutoField()

    type = TextField()
    direction = ListField()
    theta = FloatField()

    r0 = FloatField(null = True)
    L = FloatField(null = True)
    radius = FloatField(null = True)

    filletsEnabled = BooleanField(null = True)
    fillets = FloatField(null = True)
    #path = TextField()
    
    class Meta:
        database = db
        db_table = "structures"


class Mesh(Model):
    mesh_id = AutoField()
    structure_id = ForeignKeyField(Structure, backref = "meshes")

    maxSize = FloatField(null = True) 

    class Meta:
        database = db
        db_table = "meshes"
        depends_on = Structure

In [5]:
db.create_tables([Structure, Mesh])

In [22]:
import numpy

In [20]:
s1 = Structure(type = "simple", direction = [1, 0, 0], theta = 0.01)
s1.save()

m1 = Mesh(structure_id = s1, maxSize = 1e-2)
m1.save()

#db.commit()

1

In [19]:
s1.theta = 0.12
s1.save()

1

In [23]:
ss = numpy.array([
    Structure(type = "simple", direction = [1, 0, 0], theta = 0.01),
    Structure(type = "simple", direction = [1, 0, 0], theta = 0.02)
])

In [24]:
ss.dtype

dtype('O')

In [25]:
Structure.bulk_create(ss)

In [26]:
for row in Structure.select():
    print(row.structure_id, row.type, row.direction, row.theta)

1 simple [1.0, 0.0, 0.0] 0.12
2 simple [1.0, 0.0, 0.0] 0.01
3 simple [1.0, 0.0, 0.0] 0.01
4 simple [1.0, 0.0, 0.0] 0.01
5 simple [1.0, 0.0, 0.0] 0.01
6 simple [1.0, 0.0, 0.0] 0.01
7 simple [1.0, 0.0, 0.0] 0.01
8 simple [1.0, 0.0, 0.0] 0.01
9 simple [1.0, 0.0, 0.0] 0.01
10 simple [1.0, 0.0, 0.0] 0.01
11 simple [1.0, 0.0, 0.0] 0.01
12 simple [1.0, 0.0, 0.0] 0.02


In [10]:
sel = Structure.select()[-1]
sel2 = sel.meshes.select()[-1]

In [11]:
sel2

<Mesh: 8>

In [16]:
sel3 = Structure.select().dicts().get()

In [18]:
sel3

{'structure_id': 1,
 'type': 'simple',
 'direction': [1.0, 0.0, 0.0],
 'theta': 0.12,
 'r0': None,
 'L': None,
 'radius': None,
 'filletsEnabled': None,
 'fillets': None}

In [47]:
db = SqliteDatabase("newtest2.db")

In [48]:
class Test(Model):
    test_id = AutoField()
    text = TextField(null = True)
    
    class Meta:
        database = db

db.create_tables([Test])

In [49]:
db.is_closed()

False

In [50]:
test1 = Test(text = "hola")
test1.save()

1

In [51]:
test2 = Test.create(text = "asd")

In [52]:
for row in Test.select():
    print(row.test_id, row.text)

1 hola
2 asd


In [53]:
from multiprocessing import Process, Queue

In [54]:
def queue(cmd, qin, qout, *args):
    while True:
        pos, var = qin.get()
        
        if pos is None:
            break

        res = cmd(*var, *args)

        qout.put((pos, res))

    return

In [55]:
def db_save(table):
    with db.atomic():
        return table.save()

In [56]:
qin = Queue(1)
qout = Queue()
procs = []
nprocs = 10

for n in range(nprocs):
    args = (db_save, qin, qout)

    procs.append(Process(target = queue, args = args))

In [57]:
for p in procs:
    p.daemon = True
    p.start()

var = []
for n in range(50):
    var.append([Test(text = f"test_{ n }")])

for n in range(len(var)):
    qin.put((n, var[n]))

for _ in range(nprocs):
    qin.put((None, None))

results = [[] for n in range(len(var))]

for n in range(len(var)):
    index, res = qout.get() 
    results[index] = res

for p in procs:
    p.join()

In [58]:
procs[0].exitcode

0

In [59]:
test3 = Test.create(text = "afterproc")

In [60]:
for row in Test.select():
    print(row.test_id, row.text)

1 hola
2 asd
3 test_1
4 test_0
5 test_11
6 test_12
7 test_13
8 test_3
9 test_9
10 test_16
11 test_2
12 test_7
13 test_19
14 test_20
15 test_4
16 test_21
17 test_23
18 test_24
19 test_25
20 test_26
21 test_27
22 test_28
23 test_18
24 test_30
25 test_31
26 test_32
27 test_14
28 test_34
29 test_35
30 test_6
31 test_37
32 test_38
33 test_39
34 test_40
35 test_41
36 test_36
37 test_43
38 test_44
39 test_42
40 test_46
41 test_47
42 test_48
43 test_49
44 test_45
45 test_29
46 test_5
47 test_10
48 test_15
49 test_17
50 test_33
51 test_22
52 test_8
53 afterproc


In [64]:
db